In order to run this notebook, you need to install the following packages:

- `requests`
- `tqdm`
- `xarray-leaflet`

The recommended way is: `conda install -c conda-forge requests tqdm xarray-leaflet`

Note: you can replace `conda` with [mamba](https://github.com/QuantStack/mamba) if you don't like to wait ;-)

In [ ]:
import requests
import os
from tqdm import tqdm
import numpy as np
import zipfile
import xarray as xr
import xarray_leaflet
from ipyleaflet import Map, basemaps

We first download the awesome [HydroSHEDS](https://hydrosheds.org) dataset, and in particular the digital elevation model for Asia, which represents the terrain.

In [ ]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/as_dem_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = name + '/' + name + '/w001001.adf'

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

Let's open the box and see what's in there.

In [ ]:
da = xr.open_rasterio(adffile)
da

We can see that the projection is `EPSG:4326` (aka `WGS84`), which is the only projection currently supported by `xarray-leaflet`. Here the coordinate `x` corresponds to longitudes, and `y` to latitudes (in degrees). The `band` coordinate is pretty useless. The only preprocessing we will do is replacing values representing `NaN` with actual `NaN` values, which will make them transparent on the map.

In [ ]:
nan = da.attrs['nodatavals'][0]
da = da.sel(band=1)
da = xr.where(da==nan, np.nan, da)

That's it! We just need to create a map before passing it to our `DataArray` extension.

In [ ]:
m = Map(center=[40, 115], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

To show our data on the map, we call `leaflet.plot()` on our `DataArray`, and pass as parameters the map and the name of the `x` and `y` dimensions. We get back a layer, that we can further control with e.g. a slider to set the opacity.

In [ ]:
l = da.leaflet.plot(m, x_dim='x', y_dim='y')
l.interact(opacity=(0.0,1.0,0.1))